In [1]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import glob, os, csv

from langdetect import detect

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import ast
from pprint import pprint

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'http', 'javascript'])

from bs4 import BeautifulSoup

import pyLDAvis
import pyLDAvis.gensim

# import stanza
# # stanza.download('en')
# nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma')
import gc

In [2]:
df = pd.read_csv('../data/model_testing_dataset_domain_predicted.csv')
df['body_text'] = df['body_text'].apply(ast.literal_eval)
df['body_token'] = df['body_token'].apply(ast.literal_eval)
df['title'] = df['title'].apply(lambda x: str(x).strip())
df

,domain_url,title,body_text,body_token,language,predicted_class,verified_class
0,22oxht5ep3hvyboc.onion,Onion Dir - Adult,[Rape Amateur Anal Arab Asian ASMR Ass BBW Zoo...,"[rape, amateur, anal, arab, asian, asmr, bbw, ...",en,Porn,Porn
1,22pp2nrnjcmtlzja.onion,PayPal Plaza | The Tor Marketplace For Buying ...,[PayPal Plaza The Tor Marketplace For Buying S...,"[paypal, plaza, tor, marketplac, buy, sell, pa...",en,Counterfeit,Counterfeit
2,2464b3fu462tx2en.onion,CHILD PORN CENTER,"[Chika Club child porn, http 7oodcg4dnetxxegx ...","[chika, club, child, porn, oodcg, dnetxxegx, o...",en,Porn,Porn
3,24adsavjl3u6tylr.onion,"CP Video HD 2021, PTHC, Loli, child porn, pret...",[Attention the form of payment requires Javasc...,"[attent, form, payment, requir, pleas, enabl, ...",en,Porn,Porn
4,2ajtkan56n6aiyl6.onion,Hard Porn Forum,[torture man who does not listen and cries wit...,"[tortur, man, doe, listen, cri, chimpanze, enj...",en,Porn,Porn
...,...,...,...,...,...,...,...
12130,rkvwooasau2goqeamkqbade4yv2hnt6dc4ol5kgnaaoihd...,Moneys spider - financial pyramid,"[No wallet get it here https www coinbase com,...","[wallet, get, www, coinbas, com, lucki, invit,...",en,NoAccess,NoAccess
12131,e26whn2524322mkxb3cbyk27ev2ihhq2biz35hty7gzgsy...,BenTasker.co.uk - The Home of Ben Tasker - www...,[vepretty broad range of skills all built up t...,"[vepretti, broad, rang, skill, built, experi, ...",en,Services,Services
12132,4vzhn5j644aa523g.onion,Canny,"[Biggest collection with daily updates, Buy th...","[biggest, collect, daili, updat, buy, access, ...",en,Porn,Porn
12133,cashgodr53umth4z.onion,Cash God - Real Cash Sellers,"[Cash God Real Cash Sellers, We Need Your Help...","[cash, god, real, cash, seller, need, help, co...",en,Counterfeit,Counterfeit


In [5]:
df = df.groupby('title').agg({'domain_url': list, 'body_text': 'first', 'body_text': 'first', 
                             'body_token': 'first', 'language': 'first', 'predicted_class': 'first', 
                             'verified_class': 'first'})
df.reset_index(inplace=True)
df

,title,domain_url,body_text,body_token,language,predicted_class,verified_class
0,#1 Documents on Darknet,[docs7vfmtjqjkzf4fs2ugjleioylilplksdimw2gvj7lf...,[https www coinbase com Buy Bitcoins using you...,"[www, coinbas, com, buy, bitcoin, bank, accoun...",en,Counterfeit,Counterfeit
1,#youbroketheinternet (YBTI),[cheettyiapsyciew.onion],"[WE RE NOT ALONE, Partfeat Lorenzo Marsili sam...","[alon, partfeat, lorenzo, marsili, samthetechi...",en,Services,Services
2,--- ULTRAMIXER --- | Bitcoin Mixer | Bitcoin T...,[pzie5c6wt5lnrzmq.onion],"[High Volume Bitcoin Mixer, understandcan save...","[high, volum, bitcoin, mixer, understandcan, s...",en,Cryptocurrency,Cryptocurrency
3,...,[7pxhy63lo4pdhhh5.onion],[to time was as of New Years Day 2014 uncovere...,"[time, wa, new, year, day, uncov, sometim, las...",en,Services,Services
4,.Onion Links,[577eadt2m3cqooz3xngjotk66fnnmqurg4mcfbm5f6nt2...,"[Goo onion sites, Adult Porn Sex, http h4y5xra...","[goo, onion, site, adult, porn, sex, xramfioo,...",en,Services,Services
...,...,...,...,...,...,...,...
2145,���� ��������� ���������,[dmw5oa73ytwhbzit.onion],[onion],[onion],en,NoAccess,NoAccess
2146,�������-������ :: NNM-Club,"[nnmclub5toro7u65.onion, dsenxis5txr4zbxe.onion]",[Adobe Photoshop 2021 221 138 RePack by KpoJIu...,"[adob, photoshop, repack, kpojiuk, multi, ru, ...",en,Market,Market
2147,𝕃𝕒𝕨'𝕤 𝔻𝕖𝕖𝕡𝕎𝕖𝕓𝕀𝕟𝕕𝕖𝕩,[3rkyhwrfnuwsrrxozgpgmsui6pwznxg64avip5bbdm5jf...,[Nickname Password Copy Guests choosecolour Ra...,"[nicknam, password, copi, guest, choosecolour,...",en,NoAccess,NoAccess
2148,🇬🇷 🛑 🕌 – Greece Stop ISLAM — ΠΑΤΡΙΩΤΗΚΟΣ ΕΛΛΗ...,[greecee6mgg4bbhnngrb2vpcdwizjch2vq3ag4y5ag7kp...,"[Comments feed, Users TodayUsers YesterdayUser...","[comment, feed, user, todayus, yesterdayus, la...",en,Services,Services


In [6]:
df.to_csv('../data/model_testing_dataset_domain_name_predicted', index=False)